In [4]:
#空氣污染監測網  取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time

driverpath = "E:\\Python\\Anaconda\\geckodriver.exe"
browser = webdriver.Firefox(executable_path=driverpath)
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

# 模擬使用者操作行為，選擇/點擊
selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('11')  #台北市士林代號
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')

browser.find_element_by_id('ctl05_btnQuery').click()

In [5]:
# 取得資料（等瀏覽器資料出現後才執行）
html_source = browser.page_source
html_source

'<html xmlns="http://www.w3.org/1999/xhtml" lang="zh-TW"><head id="Head1"><base href="https://taqm.epa.gov.tw/taqm/tw/"><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><title>行政院環境保護署－空氣品質監測網 - 月平均值查詢</title><meta name="title" content="空氣品質監測網(TAQM)">\n<!--link rel="govtwsimpledc.xsd" href="http://www.gov.tw/schema/dc/" title="Dublin Core" /--> \n<link type="text/css" href="/taqm/tw/style/all.min.css?v=20180508c" rel="stylesheet">\n<script type="text/javascript" async="" src="https://www.google-analytics.com/analytics.js"></script><script type="text/javascript" src="/taqm/script/jquery.min.js"></script>\n<script type="text/javascript" src="/taqm/script/jquery.myslide2.min.js"></script>\n<script type="text/javascript">\nfunction DownloadReport(url) {\n\tvar win = window.open("DownloadReport.aspx?file="+escape(url),"download" /*,"modal=yes,menubar=no,scrollbars=no,status=no,titlebar=no,toolbar=no,height=200px,width=3

In [6]:
# 關閉瀏覽器
browser.close()

In [7]:
soup = BeautifulSoup(html_source, 'html.parser')
table = soup.find('table', class_='TABLE_G')
table

<table align="Center" border="1" cellpadding="3" cellspacing="0" class="TABLE_G" id="ctl05_gv" rules="all" style="border-color:Black;border-width:1px;border-style:None;border-collapse:collapse;">
<tbody><tr style="color:Black;font-weight:normal;">
<th scope="col">監測項目</th><th scope="col">單位</th><th scope="col">監測日期</th><th scope="col">監測值</th><th scope="col">標註</th>
</tr><tr style="color:Black;">
<td class="no-alt" rowspan="12" style="white-space:nowrap;" valign="top">SO2</td><td class="no-alt" rowspan="12" valign="top">ppb</td><td>2018/01</td><td>1.80</td><td> </td>
</tr><tr class="ALT" style="color:Black;">
<td>2018/02</td><td>1.90</td><td> </td>
</tr><tr style="color:Black;">
<td>2018/03</td><td>2.20</td><td> </td>
</tr><tr class="ALT" style="color:Black;">
<td>2018/04</td><td>2.30</td><td> </td>
</tr><tr style="color:Black;">
<td>2018/05</td><td>3.10</td><td> </td>
</tr><tr class="ALT" style="color:Black;">
<td>2018/06</td><td>2.70</td><td> </td>
</tr><tr style="color:Black;">
<td>

In [12]:
data ={}

for tr in table.find_all('tr')[1:]:
    for i, td in enumerate(tr.find_all('td')):
        #依選項的字串長度作判斷
        if len(tr.find_all('td')) == 5:
            if i == 0:  #column name
                obs = td.text                
                data.setdefault(obs, {})
            if i == 2:  #date
                date = td.text
            if i == 3:  #value
                value = td.text
                data[obs][date] = value
                
        if len(tr.find_all('td')) == 3:
            if i == 0:
                date = td.text
            if i == 1:
                value = td.text
                data[obs][date] = value
print(data)

{'SO2': {'2018/01': '1.80', '2018/02': '1.90', '2018/03': '2.20', '2018/04': '2.30', '2018/05': '3.10', '2018/06': '2.70', '2018/07': '2.20', '2018/08': '2.40', '2018/09': '2.10', '2018/10': '1.70', '2018/11': '1.90', '2018/12': '1.80'}, 'CO': {'2018/01': '0.34', '2018/02': '0.44', '2018/03': '0.40', '2018/04': '0.38', '2018/05': '0.34', '2018/06': '0.29', '2018/07': '0.21', '2018/08': '0.30', '2018/09': '0.26', '2018/10': '0.29', '2018/11': '0.30', '2018/12': '0.35'}, 'O3': {'2018/01': '33.40', '2018/02': '32.50', '2018/03': '35', '2018/04': '38.40', '2018/05': '31.60', '2018/06': '29.50', '2018/07': '18.70', '2018/08': '26.40', '2018/09': '29.10', '2018/10': '45.90', '2018/11': '32.40', '2018/12': '30.70'}, 'PM10': {'2018/01': '23', '2018/02': '41', '2018/03': '39', '2018/04': '48', '2018/05': '37', '2018/06': '26', '2018/07': '24', '2018/08': '26', '2018/09': '28', '2018/10': '33', '2018/11': '25', '2018/12': '21'}, 'NOx': {'2018/01': '14.17', '2018/02': '19.43', '2018/03': '18.50',

In [13]:
import pandas as pd

# 取出題目要求的區間
print('===== 取出 台北市士林區 2018 的 SO2 資料 =====')
print(data['SO2'])

print('===== 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料 =====')
print(dict([(key, value) for key, value in data['SO2'].items() if key < '2018/09' ]))

# 整理成 DataFrame
d_SO2 = [{'時間': key, '數值': value} for key, value in data['SO2'].items() if key < '2018/09' ]
pd.DataFrame(d_SO2)

===== 取出 台北市士林區 2018 的 SO2 資料 =====
{'2018/01': '1.80', '2018/02': '1.90', '2018/03': '2.20', '2018/04': '2.30', '2018/05': '3.10', '2018/06': '2.70', '2018/07': '2.20', '2018/08': '2.40', '2018/09': '2.10', '2018/10': '1.70', '2018/11': '1.90', '2018/12': '1.80'}
===== 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料 =====
{'2018/01': '1.80', '2018/02': '1.90', '2018/03': '2.20', '2018/04': '2.30', '2018/05': '3.10', '2018/06': '2.70', '2018/07': '2.20', '2018/08': '2.40'}


,時間,數值
0,2018/01,1.80
1,2018/02,1.90
2,2018/03,2.20
3,2018/04,2.30
4,2018/05,3.10
5,2018/06,2.70
6,2018/07,2.20
7,2018/08,2.40
